# FEniCSx H and P Tutorial

In this tutorial, we will be continuing our FEniCSx Bridge example, except this time we will be fully incoprorating the 3rd dimension and make our bridge a 3D structure. Last time, our bridge had a force applied to it at a single point at the center of the beam but in the Y direction. This time, we added depth and will be applying a downward force in the Z-axis. Before getting into the problem, please ensure that FEniCSx and DolfinX are installed on your computer. For assistance with installation, please refer to the `readme.md` for tips on installing the platform, creating a conda environment, activating it (also in your terminal/visual studio session), and running the Jupyter Notebook.

## Library Imports

To ensure that FEniCSx runs properly, please run each Jupyter notebook cell in order. If you run any cells ahead of others, the code might error out. First and foremost, we call our imports, the main ones being Numpy, DolfinX, UFL. 

Here:

- Numpy:        Standard import which is for computations and arrays
- DolfinX:      Core FEniCSx library for FEA
- UFL:          Domain-specific language usedfor FEA


In [ ]:
import numpy as np
from mpi4py import MPI
from dolfinx import mesh, fem, plot
import ufl
import pyvista
from dolfinx.fem import petsc
from dolfinx.plot import vtk_mesh

## Simple Bridge Problem Setup

Next we need to initialize our problem variables. As discussed before, we are setting up a 3 dimensional bridge, fixed on both ends, and simulating a downward force right along the beam. Differing from part 1, we are no longer applying a force as a point right in the middle of the bridge, instead we are applying a uniform load across the whole bridge. This should provide a good visualization distinction from what we saw previously. 

The steps for this problem will look like this:
1) Define mesh
2) Create the domain space (label the edges)
3) Define material properties

In [ ]:
# Bridge dimensions
L, W, H = 10.0, 1.0, 0.5  # Length, width, height
num_elements = [50, 5, 5]  # Mesh resolution 

# Create a 3D bridge mesh
domain = mesh.create_box(MPI.COMM_WORLD, [[0.0, 0.0, 0.0], [L, W, H]], num_elements, mesh.CellType.hexahedron)
V = fem.functionspace(domain, ("Lagrange", 1, (domain.geometry.dim, )))

# Material properties for bridge
rho = 7850  # Density 
E = default_scalar_type(2.1e11)  
nu = default_scalar_type(0.3)
mu = fem.Constant(domain, E / (2 * (1 + nu)))
lmbda = fem.Constant(domain, E * nu / ((1 + nu) * (1 - 2 * nu)))


In [ ]:
# Define the weak form
u = fem.Function(V)  # Displacement
v = ufl.TestFunction(V) # Test function
u_t = fem.Function(V)  # Velocity
u_tt = fem.Function(V)  # Acceleration

I = ufl.variable(ufl.Identity(domain.geometry.dim)) 
F = ufl.variable(I + ufl.grad(u)) 
C = ufl.variable(F.T * F) 
Ic = ufl.variable(ufl.tr(C)) 
J = ufl.variable(ufl.det(F)) 
psi = (mu / 2) * (Ic - 3) - mu * ufl.ln(J) + (lmbda / 2) * (ufl.ln(J))**2
P = ufl.diff(psi, F) 

## Boundary Conditions

Simple boundary condition applications where we define where the left and right boundaries of the beam are. Left boundary conditions are at 0,0. Right boundary conditions are the length of the beam.

In [ ]:
left_dofs = fem.locate_dofs_geometrical(V, left_end)
right_dofs = fem.locate_dofs_geometrical(V, right_end)
zero_displacement = np.array([0.0, 0.0, 0.0], dtype=default_scalar_type)
bcs = [fem.dirichletbc(zero_displacement, left_dofs, V),
       fem.dirichletbc(zero_displacement, right_dofs, V)]

## Force Application

Next we define our downward force for our beam. There are many different ways to do this, but we are using a uniform constant downward Z force. 

In [ ]:
# External force: Vertical force applied at the center of the bridge
force_center = fem.Constant(domain, default_scalar_type((0.0, 0.0, -1e4)))  # Force in the negative z-direction
ds = ufl.Measure("ds", domain=domain)
dx = ufl.Measure("dx", domain=domain)
F_form = rho * ufl.dot(u_tt, v) * dx + ufl.inner(ufl.grad(v), P) * dx - ufl.dot(v, force_center) * ds

## Solver Initialization

We now need to initialize our solver and set up the tolerance needed for our computation to be considered "close enough". 

In [ ]:
# Solver setup
problem = NonlinearProblem(F_form, u, bcs)
solver = NewtonSolver(domain.comm, problem)
solver.atol = 1e-8
solver.rtol = 1e-8
solver.convergence_criterion = "incremental"

## Visualization

We are going to visualize this in two ways. The first by creating a grid of the mesh itself, and then plotting the mesh. Note that we are not using the standard pyplot functions as we are not using that library at all. FEniCSx works incredibly well when using the pyvista library, as it specializes in vector and array plotting, handling 3 dimensional simulation with ease.

In [ ]:
pyvista.start_xvfb()
plotter = pyvista.Plotter()
plotter.open_gif("bridge_dynamics_h_mesh.gif", fps=10)

topology, cells, geometry = plot.vtk_mesh(u.function_space)
function_grid = pyvista.UnstructuredGrid(topology, cells, geometry)
function_grid["u"] = np.zeros((geometry.shape[0], 3))
function_grid.set_active_vectors("u")

actor = plotter.add_mesh(function_grid, show_edges=True, lighting=False, clim=[0, 0.1])

## Time Steps

Finally, because we are working with a non-linear problem, we need to take into account how accelaration and velocity changes based on what time step we are on. Usually for linear problems, it would be a constant application, but this is where we utilize Fenicsx to its fullest and show how it can solve complex computations. 

In [ ]:
# Time-stepping loop
log.set_log_level(log.LogLevel.INFO)
for step in range(num_steps):
    t = step * dt
    print(f"Time step {step + 1}/{num_steps}, Time: {t:.2f}s")
    
    # Solve for the displacement
    num_its, converged = solver.solve(u)
    assert converged, f"Solver did not converge at step {step + 1}"
    u.x.scatter_forward()

    # Update the velocity and acceleration for the next time step
    u_t.x.array[:] = u.x.array[:]
    u_tt.x.array[:] = (u.x.array - u_t.x.array) / dt
    function_grid["u"] = u.x.array[:].reshape(-1, 3)
    function_grid["u"] = np.zeros((geometry.shape[0], 3))
    function_grid.set_active_vectors("u")
    plotter.update_coordinates(function_grid, render=False)
    plotter.write_frame()

plotter.close()

## Results

We have a saved `bridge_dynamics_h_mesh.gif` file saved in the same directory. This shows us the deformation that occurs on our bridge when we apply a force in the downward Z direction. In comparison to part 1, we see how our structure now deforms downward as opposed to "out of the screen" in the Y direction.